In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
import numpy as np
import pickle
import json
from datetime import datetime
import warnings
from matplotlib import pyplot as plt
import seaborn as sns
from nilearn.plotting import plot_design_matrix
from nilearn.image import load_img
from nilearn.reporting import make_glm_report
from nilearn import image
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix
sys.path.append('..')
from utils.data import Subject, load_participant_list
from utils.analysis import compute_parametric_modulator

In [4]:
base_dir = '/home/ubuntu/data/learning-habits'
bids_dir = "/home/ubuntu/data/learning-habits/bids_dataset/derivatives/fmriprep-24.0.1"

sub_ids = load_participant_list(base_dir)

In [5]:
#subjects = [Subject(base_dir, sub_id, include_modeling=True, include_imaging=True, bids_dir=bids_dir) for sub_id in sub_ids]
subject = Subject(base_dir, '01', include_modeling=True, include_imaging=True, bids_dir=bids_dir)

In [6]:
model_params = {
    'model_name': 'no_modulation',
    'tr': 2.33384,
    'hrf_model': 'spm',
    'noise_model': 'ar1',
    'smoothing_fwhm': 5,
    'high_pass': 0.01,
    'motion_type': 'basic',
    'include_physio': True,
    'brain_mask': False,
    'mask_samples': False,
    'demean_modulator': True
}

run = 'learning2'

In [28]:
# Parameters
model_name = model_params["model_name"]
tr = model_params["tr"]
hrf_model = model_params["hrf_model"]
noise_model = model_params["noise_model"]
smoothing_fwhm = model_params["smoothing_fwhm"]
high_pass = model_params["high_pass"]
motion_type = model_params["motion_type"]
include_physio = model_params["include_physio"]
brain_mask = model_params["brain_mask"]
mask_samples = model_params["mask_samples"]
demean_modulator = model_params["demean_modulator"]

# Load confounds
confounds, sample_mask = subject.load_confounds(run, motion_type=motion_type)
if include_physio:
    physio_regressors = subject.load_physio_regressors(run)
    confounds = confounds.join(physio_regressors)

if not mask_samples:
    sample_mask = None

# Load fMRI volume
img_path = subject.img.get(run)
fmri_img = load_img(img_path)

TypeError: Subject.load_confounds() got an unexpected keyword argument 'strategy'

In [27]:
confounds.columns

Index(['csf', 'rot_x', 'rot_y', 'rot_z', 'trans_x', 'trans_y', 'trans_z',
       'white_matter', 'cosine00', 'cosine01', 'cosine02', 'cosine03',
       'cosine04', 'physio1', 'physio2', 'physio3', 'physio4', 'physio5',
       'physio6', 'physio7', 'physio8', 'physio9', 'physio10', 'physio11',
       'physio12', 'physio13', 'physio14', 'physio15', 'physio16', 'physio17',
       'physio18'],
      dtype='object')

In [8]:
# This should always be None for now
if brain_mask:
    brain_mask_path = subject.brain_mask.get(run)
    brain_mask = load_img(brain_mask_path)
else:
    brain_mask = None

n = fmri_img.shape[-1]
frametimes = np.linspace(tr / 2., (n - .5) * tr, n)

In [9]:
columns_event = {'first_stim_value_rl':'first_stim_presentation',
                'first_stim_value_ck':'first_stim_presentation',
                'first_stim':'first_stim_presentation'}

# Load events
events = getattr(subject, run).extend_events_df(columns_event)

In [10]:
events['trial_type'] = events.apply(
    lambda row: f"{row['trial_type']}_{'exclude' if int(row['first_stim']) in (1, 8) else 'include'}"
    if row['trial_type'] == 'first_stim_presentation' else row['trial_type'],
    axis=1
)

In [11]:
events

,onset,duration,trial_type,trial,first_stim_value_rl,first_stim_value_ck,first_stim
0,0.001868,0.820289,first_stim_presentation_include,1,3.988659,0.101242,7.0
1,0.822156,1.000000,second_stim_presentation,1,0.000000,0.000000,0.0
2,1.822156,1.504399,non_response_feedback,1,0.000000,0.000000,0.0
3,3.326556,6.266152,iti,1,0.000000,0.000000,0.0
4,9.592707,0.848439,first_stim_presentation_exclude,2,1.003978,0.043020,1.0
...,...,...,...,...,...,...,...
567,959.462614,0.623545,second_stim_presentation,96,0.000000,0.000000,0.0
568,960.086159,0.000000,response,96,0.000000,0.000000,0.0
569,960.087956,1.879011,purple_frame,96,0.000000,0.000000,0.0
570,960.589006,1.377960,points_feedback,96,0.000000,0.000000,0.0


In [12]:
# Ignore warnings related to null duration events and unexpected columns in events data
warnings.filterwarnings("ignore", message=".*events with null duration.*")
warnings.filterwarnings("ignore", message=".*following unexpected columns in events data.*")
# Create design matrix
design_matrix = make_first_level_design_matrix(frame_times=frametimes,
                                    events=events,
                                    hrf_model=hrf_model,
                                    drift_model=None,
                                    high_pass=high_pass,
                                    add_regs=confounds)

In [13]:
demean_modulator = 'zscore'

In [14]:
# RL Parametric modulation
parametric_modulator_column = 'first_stim_value_rl'
condition = 'first_stim_presentation_include'
reg_value = compute_parametric_modulator(events, condition, parametric_modulator_column,
                                            frametimes, hrf_model, normalize=demean_modulator)
design_matrix.insert(1, parametric_modulator_column, reg_value)

In [15]:
# CK Parametric modulation
parametric_modulator_column = 'first_stim_value_ck'
condition = 'first_stim_presentation_include'
reg_value = compute_parametric_modulator(events, condition, parametric_modulator_column,
                                            frametimes, hrf_model, normalize=demean_modulator)
design_matrix.insert(2, parametric_modulator_column, reg_value)

In [17]:
# plt.figure(figsize=(18, 6))
# plt.plot(design_matrix['first_stim_value_ck'][:300], label='first_stim_value_ck')
# plt.plot(design_matrix['first_stim_value_rl'][:300], label='first_stim_value_rl')
# plt.plot(design_matrix['first_stim_presentation_include'][:300], label='first_stim_presentation_include')
# plt.plot(design_matrix['first_stim_presentation_exclude'][:300], label='first_stim_presentation_exclude')
# plt.legend()
# plt.show()

In [18]:
np.corrcoef(design_matrix['first_stim_value_ck'], design_matrix['first_stim_value_rl'])

array([[1.        , 0.13847168],
       [0.13847168, 1.        ]])

In [ ]:
# Compute the average correlation matrix
# average_correlation_matrix = design_matrix.corr().values
# plt.figure(figsize=(30, 28))
# sns.heatmap(average_correlation_matrix, xticklabels=design_matrix.columns, 
#             yticklabels=design_matrix.columns, cmap='coolwarm', annot=True)
# plt.title('Average Correlation Matrix')
# plt.show()

In [24]:
design_matrix.columns

Index(['first_stim_presentation_exclude', 'first_stim_value_rl',
       'first_stim_value_ck', 'first_stim_presentation_include', 'iti',
       'non_response_feedback', 'points_feedback', 'purple_frame', 'response',
       'second_stim_presentation', 'csf', 'rot_x', 'rot_y', 'rot_z', 'trans_x',
       'trans_y', 'trans_z', 'white_matter', 'cosine00', 'cosine01',
       'cosine02', 'cosine03', 'cosine04', 'physio1', 'physio2', 'physio3',
       'physio4', 'physio5', 'physio6', 'physio7', 'physio8', 'physio9',
       'physio10', 'physio11', 'physio12', 'physio13', 'physio14', 'physio15',
       'physio16', 'physio17', 'physio18', 'constant'],
      dtype='object')

In [25]:
design_matrix.colinearity()

AttributeError: 'DataFrame' object has no attribute 'colinearity'